# DATA TRANSFORMATION - 10_urgencias
## 0. Previos

Se cargan las bases y las librerías a utilizar.

In [1]:
# Librerías
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import numpy as np

import generic_funcions as gf

ruta_archivos = 'C:/Users/monic/documentos/ds4a/project/datos/'

## 1. Load

In [2]:
dcc = gf.diccionario_llaves()
dcc

{'ACT': {'id': 'ID',
  'fecha': 'FE_RESULTADO',
  'fecha_no_ok': False,
  'prefi': 'act'},
 'ACT_DESAGREGADO': {'id': 'NUMERO IDENTIFICACION',
  'fecha': 'FE_RESULTADO',
  'fecha_no_ok': False,
  'prefi': 'acd'},
 'Adherencia': {'id': 'ds_identificacion',
  'fecha': 'FE_ENTREVISTA',
  'fecha_no_ok': False,
  'prefi': 'adh'},
 'Antecedentes_familiares': {'id': 'Id',
  'fecha': 'FE_ALTA',
  'fecha_no_ok': False,
  'prefi': 'ant'},
 'Antecedentes_patologicos': {'id': 'DS_IDENTIFICACION',
  'fecha': 'FE_ACTUALIZA',
  'fecha_no_ok': False,
  'prefi': 'ant'},
 'Ayudas_diagnosticas': {'id': 'Numero_Identificacion',
  'fecha': 'Fecha_Orden',
  'fecha_no_ok': False,
  'prefi': 'ayu'},
 'Biologicos Asma': {'id': 'Identificacion',
  'fecha': 'Fecha_Dcto',
  'fecha_no_ok': False,
  'prefi': 'bio'},
 'Calidad de vida relacioada en salud': {'id': 'Identificacion',
  'fecha': 'FE_ALTA',
  'fecha_no_ok': False,
  'prefi': 'cal'},
 'Disnea': {'id': 'id',
  'fecha': 'FE_ALTA',
  'fecha_no_ok': False,
  

In [3]:
modulo = 'Urgencias'

In [4]:
base = gf.carga_datos( ruta = ruta_archivos, diccionario = dcc, modulo = modulo )
base.head()

,id,fecha,Regional_EPS_Desc,Codigo_Prestacion_Op,Descripcion_Prestacion,Codigo_Diagnostico_EPS_Op,Diagnostico_EPS_Desc,Cantidad_Autorizada,year,month,year_month
0,1759148,2019-09-29,Pereira,S0050115,ATENCION MEDICA PRIORITARIA,Z759,PROBLEMA NO ESPECIFICADO RELACIONADO CON SERVI...,1,2019,9,201909
1,1110797,2019-07-20,Armenia,S0050115,ATENCION MEDICA PRIORITARIA,R104,OTROS DOLORES ABDOMINALES Y LOS NO ESPECIFICADOS,1,2019,7,201907
2,1497880,2019-07-21,Armenia,S0050115,ATENCION MEDICA PRIORITARIA,Z759,PROBLEMA NO ESPECIFICADO RELACIONADO CON SERVI...,1,2019,7,201907
3,1598583,2019-06-18,Pereira,S0050115,ATENCION MEDICA PRIORITARIA,Z759,PROBLEMA NO ESPECIFICADO RELACIONADO CON SERVI...,1,2019,6,201906
4,525540,2019-06-11,Pereira,S0050115,ATENCION MEDICA PRIORITARIA,Z759,PROBLEMA NO ESPECIFICADO RELACIONADO CON SERVI...,1,2019,6,201906


## 2. Transform

* Kind of service

In [5]:
base.groupby('Descripcion_Prestacion')['id'].count()

Descripcion_Prestacion
ATENCION MEDICA PRIORITARIA              3146
CONSULTA MEDICO GENERAL NO PROGRAMADA    1940
Name: id, dtype: int64

In [7]:
variable = 'service_cat'
base[variable] = base['Descripcion_Prestacion'].apply( lambda x : 'otra' if 'CONSULTA' in x else 'urg' )
base.groupby(variable)['id'].count()

service_cat
otra    1940
urg     3146
Name: id, dtype: int64

* Kind of disease

In [8]:
base = gf.letra_codigo( base, 'Codigo_Diagnostico_EPS_Op' )
base.head()

,id,fecha,Regional_EPS_Desc,Codigo_Prestacion_Op,Descripcion_Prestacion,Codigo_Diagnostico_EPS_Op,Diagnostico_EPS_Desc,Cantidad_Autorizada,year,month,year_month,service_cat,Codigo_Diagnostico_EPS_Op_cod
0,1759148,2019-09-29,Pereira,S0050115,ATENCION MEDICA PRIORITARIA,Z759,PROBLEMA NO ESPECIFICADO RELACIONADO CON SERVI...,1,2019,9,201909,urg,z
1,1110797,2019-07-20,Armenia,S0050115,ATENCION MEDICA PRIORITARIA,R104,OTROS DOLORES ABDOMINALES Y LOS NO ESPECIFICADOS,1,2019,7,201907,urg,r
2,1497880,2019-07-21,Armenia,S0050115,ATENCION MEDICA PRIORITARIA,Z759,PROBLEMA NO ESPECIFICADO RELACIONADO CON SERVI...,1,2019,7,201907,urg,z
3,1598583,2019-06-18,Pereira,S0050115,ATENCION MEDICA PRIORITARIA,Z759,PROBLEMA NO ESPECIFICADO RELACIONADO CON SERVI...,1,2019,6,201906,urg,z
4,525540,2019-06-11,Pereira,S0050115,ATENCION MEDICA PRIORITARIA,Z759,PROBLEMA NO ESPECIFICADO RELACIONADO CON SERVI...,1,2019,6,201906,urg,z


In [54]:
base_ = base[base['Cantidad_Autorizada'] != '?']
base_['cant'] = base_['Cantidad_Autorizada'].astype( int )
base_['diag'] = base_['Codigo_Diagnostico_EPS_Op_cod'].apply( lambda x : 'j' if x == 'j' else 'otra' )

ids = ['id','year','month']
variables = ['service_cat','diag','cant']

base_ = base_[ ids + variables]

base_.head()

<ipython-input-54-ff91cd522603>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_['cant'] = base_['Cantidad_Autorizada'].astype( int )
<ipython-input-54-ff91cd522603>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_['diag'] = base_['Codigo_Diagnostico_EPS_Op_cod'].apply( lambda x : 'j' if x == 'j' else 'otra' )


,id,year,month,service_cat,diag,cant
0,1759148,2019,9,urg,otra,1
1,1110797,2019,7,urg,otra,1
2,1497880,2019,7,urg,otra,1
3,1598583,2019,6,urg,otra,1
4,525540,2019,6,urg,otra,1


In [67]:
# pivot table to have separate variables, nans replaced with zero

# total
base_p = base_.pivot_table(index=ids, columns=['service_cat'], values='cant', aggfunc=np.sum).reset_index().fillna(0)

base_p['total'] = base_p['urg'] + base_p['otra']

base_p = base_p[ ids  + ['urg','total'] ]
pre = dcc[modulo]['prefi'] + '_' 
base_p.columns = [ pre + s for s in base_p.columns]
base_p.rename( columns={ pre + 'id':'id', pre + 'year':'year', pre + 'month':'month' }, inplace=True)

base_p.head()

,id,year,month,urg_urg,urg_total
0,500547,2017,2,1.0,1.0
1,500547,2018,12,1.0,1.0
2,500547,2019,1,0.0,2.0
3,500547,2019,6,0.0,4.0
4,500547,2020,2,1.0,1.0


In [70]:
# Related with athsma

base_j = base_[base_['diag']=='j']
base_p_j = base_j.pivot_table(index=ids, columns=['service_cat'], values='cant', aggfunc=np.sum).reset_index().fillna(0)
base_p_j['total'] = base_p_j['urg'] + base_p_j['otra']

base_p_j = base_p_j[ ids  + ['urg','total'] ]
pre = dcc[modulo]['prefi'] + '_j_' 
base_p_j.columns = [ pre + s for s in base_p_j.columns]
base_p_j.rename( columns={ pre + 'id':'id', pre + 'year':'year', pre + 'month':'month' }, inplace=True)

base_p_j.head()

,id,year,month,urg_j_urg,urg_j_total
0,506808,2020,3,1.0,1.0
1,525295,2018,11,2.0,2.0
2,525295,2018,12,2.0,2.0
3,525295,2019,1,3.0,5.0
4,525295,2019,5,3.0,5.0


In [74]:
# Join
base_t = base_p.merge( base_p_j, how = 'left' )
base_t = base_t.fillna( 0 )
base_t.head()

,id,year,month,urg_urg,urg_total,urg_j_urg,urg_j_total
0,500547,2017,2,1.0,1.0,0.0,0.0
1,500547,2018,12,1.0,1.0,0.0,0.0
2,500547,2019,1,0.0,2.0,0.0,0.0
3,500547,2019,6,0.0,4.0,0.0,0.0
4,500547,2020,2,1.0,1.0,0.0,0.0


## 3. Merge with ids table

In [75]:
ids_mensual = gf.base_ids_mensual( ruta_archivos )
base_final_hos = ids_mensual.merge( base_t, how='left')
base_final_hos.head()

,id,year,month,urg_urg,urg_total,urg_j_urg,urg_j_total
0,500547,2016,1,NaN,NaN,NaN,NaN
1,500547,2016,2,NaN,NaN,NaN,NaN
2,500547,2016,3,NaN,NaN,NaN,NaN
3,500547,2016,4,NaN,NaN,NaN,NaN
4,500547,2016,5,NaN,NaN,NaN,NaN


In [76]:
base_final_hos.shape

(53280, 7)